### python logging

In [3]:
import logging
from datetime import datetime


def start_logger():
    log_file = f'daily_report_{datetime.now().strftime("%m%d%Y_%H%M%S")}.log'
    logging.basicConfig(filename=log_file,
                        level=logging.DEBUG,
                        format='%(asctime)s %(message)s',
                        datefmt='%m-%d %H:%M:%S')

def main():
    start_logger()
    logging.debug("SCRIPT: I'm starting to do things!")

    try:
        20 / 0
    except Exception:
        logging.exception('SCRIPT: We had a problem!')
        logging.error('SCRIPT: Issue with division in the main() function')

    logging.debug('SCRIPT: About to wrap things up!')


main()

### Adding Automated Messaging

### Mail

In [4]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import os
import configparser

def get_config(env):
    config = configparser.ConfigParser()
    if env == "DEV":
        config.read(['config/development.cfg'])
    elif env == "PROD":
        config.read(['config/production.cfg'])
    return config

def mail(to, subject, text, attach=None, config=None):
    if not config:
        config = get_config("DEV")
    msg = MIMEMultipart()
    msg['From'] = config.get('email', 'user')
    msg['To'] = ", ".join(to)
    msg['Subject'] = subject
    msg.attach(MIMEText(text))
    if attach:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(open(attach, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',
                        'attachment; filename="%s"' % os.path.basename(attach))
        msg.attach(part)
    mailServer = smtplib.SMTP("smtp.gmail.com", 587)
    mailServer.ehlo()
    mailServer.starttls()
    mailServer.ehlo()
    mailServer.login(config.get('email', 'user'),
                     config.get('email', 'password'))
    mailServer.sendmail(config.get('email', 'user'), to, msg.as_string())
    mailServer.close()

def example():
    mail(['listof@mydomain.com', 'emails@mydomain.com'],
         "Automate your life: sending emails",
         "Why'd the elephant sit on the marshmallow?",
         attach="my_file.txt")


### SMS and voice

In [6]:
from twilio.rest import Client
import configparser

def send_text(sender, recipient, text_message, config=None):
    if not config:
        config = configparser.ConfigParser()
        config.read('config/development.cfg')

    client = Client(config.get('twilio', 'account_sid'),
                    config.get('twilio', 'auth_token'))
    sms = client.messages.create(body=text_message,
                                 to=recipient,
                                 from_=sender)

def example():
    send_text("+11008675309", "+11088675309", "JENNY!!!!")

### Chat integration

In [9]:
from hypchat import HypChat

def get_config(env):
    config = configparser.ConfigParser()
    if env == "DEV":
        config.read(['config/development.cfg'])
    elif env == "PROD":
        config.read(['config/production.cfg'])
    return config

def get_client(config):
    client = HypChat(config.get('hipchat', 'token'))
    return client

def message_room(client, room_name, message):
    try:
        room = client.get_room(room_name)
        room.message(message)
    except Exception as e:
        print(e)

def main():
    config = get_config('DEV')
    client = get_client(config)
    message_room(client, 'My Favorite Room', "I'M A ROBOT!")